Complimentary to bert model, use other features to do predictions

In [1]:
import numpy as np
import time
import json
import os
import pandas as pd
import re


In [6]:
## Load data

TRAIN_DATA_PATH = "../full_data/data_storage/full_dev_train.json"
Y_TRAIN_ID_PATH = "../id_data/train.label.txt"
X_TRAIN_ID_PATH = "../id_data/train.data.txt"   # train

DEV_DATA_PATH = "../full_data/data_storage/full_dev_train.json"
Y_DEV_ID_PATH = "../id_data/dev.label.txt"
X_DEV_ID_PATH = "../id_data/dev.data.txt"   # dev

TEST_DATA_PATH = "../full_data/data_storage/test_full_v2.json"
X_TEST_ID_PATH = "../id_data/test.data.txt"   # test

In [17]:
def join_data_id_label_user(LABEL_PATH, ID_PATH, DATA_PATH, dropna_on_column="text", SOURCE_STORY_ONLY=True):
    """A function that joins data and id and their labels, return a dataframe that trims off twitter we don't have data"""
    # Process Labels
    if LABEL_PATH:
        # 1: Rumour
        # 0: NonRumour
        with open(LABEL_PATH, "r") as f:
            y_label = f.read().strip().split("\n") # remove next line
        y_label = pd.DataFrame(y_label, columns = ["label"])
        y_label[y_label["label"]=="rumour"] = 1
        y_label[y_label["label"]=="nonrumour"] = 0
    
    
    ## Get Dataframe Id
    total_id_list = []
    with open(ID_PATH, "r") as f:
        for line in f:
            line = line.strip() # remove next line
            if SOURCE_STORY_ONLY:
                line = line.split(',')[0] # split into list\
            else:
                line = line.split(',')
            total_id_list.append(line)
            
    if not SOURCE_STORY_ONLY:
        total_id_list = [item for sublist in total_id_list for item in sublist] # Flat into a single list
    total_id_list = total_id_list
    id_df = pd.DataFrame(total_id_list, columns=["id"])
    id_df["id"] = id_df["id"].astype(str)
    # join their labels
    if LABEL_PATH:
    # if we are processing testing data, don't add label
        id_df["label"] = y_label["label"]
    
    ## Get Twitter Data
    # Read Source Data
    data_df = pd.read_json(DATA_PATH)
    #source_df = pd.concat([source_df, y_train], axis=1)
    data_df["id"] = data_df["id"].astype(str)
    
    ## Join the data file and IDs we got 
    ## Drop rows we don't have data on
    combined_df = id_df.join(data_df.set_index('id'), on='id')
    #if dropna_on_column:
        #combined_df = combined_df.dropna(subset=['text', 'author_id']).reset_index(drop=True)
    combined_df.reindex(total_id_list) # make sure we have the right order
    if LABEL_PATH:
        labels = combined_df["label"].astype(int)
    combined_df = pd.json_normalize(combined_df['user'])
    if LABEL_PATH:
        combined_df["label"] = labels
    combined_df.loc[combined_df["translator_type"]=="none", ["translator_type"]] = 0
    combined_df.loc[combined_df["translator_type"]!="none", ["translator_type"]] = 1
    combined_df.loc[combined_df["verified"]==True, ["verified"]] = 1
    combined_df.loc[combined_df["verified"]!=True, ["verified"]] = 0
    combined_df.loc[combined_df["default_profile"]==True, ["default_profile"]] = 1
    combined_df.loc[combined_df["default_profile"]!=True, ["default_profile"]] = 0
    if LABEL_PATH:
        return combined_df[['followers_count', 'friends_count', 'listed_count','favourites_count', 'verified', 'statuses_count',
           'default_profile', 'translator_type', 'label']]
    else:
        return combined_df[['followers_count', 'friends_count', 'listed_count','favourites_count', 'verified', 'statuses_count',
           'default_profile', 'translator_type']]

In [8]:
train_df = join_data_id_label_user(Y_TRAIN_ID_PATH, X_TRAIN_ID_PATH, TRAIN_DATA_PATH)
train_df.dropna(inplace=True)
train_df

,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,default_profile,translator_type,label
0,410.0,2395.0,1.0,34431.0,0,26608.0,1,1,0
1,3231508.0,884.0,20722.0,374.0,1,480887.0,0,1,1
2,612.0,951.0,1.0,25405.0,0,7130.0,0,1,0
3,19812480.0,1065.0,125319.0,1151.0,1,371093.0,0,1,0
4,4880.0,5117.0,6.0,21874.0,0,13439.0,1,1,0
...,...,...,...,...,...,...,...,...,...
1889,630.0,640.0,0.0,106385.0,0,32591.0,0,1,0
1890,143082.0,58282.0,1264.0,59928.0,1,54722.0,0,1,1
1891,17463344.0,488.0,65447.0,44.0,1,384224.0,0,1,1
1892,9088626.0,1857.0,50490.0,780.0,1,337589.0,0,1,0


In [9]:
dev_df = join_data_id_label_user(Y_DEV_ID_PATH, X_DEV_ID_PATH, DEV_DATA_PATH)
dev_df.fillna(0, inplace=True)
dev_df

,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,default_profile,translator_type,label
0,35334.0,3522.0,984.0,2491.0,0,64580.0,0,1,0
1,757.0,1096.0,6.0,44173.0,0,14321.0,0,1,0
2,3004.0,2102.0,30.0,6011.0,0,111198.0,0,1,0
3,935.0,2554.0,28.0,36123.0,0,17300.0,0,1,0
4,1366579.0,770.0,9451.0,9348.0,1,47011.0,0,1,0
...,...,...,...,...,...,...,...,...,...
627,113.0,867.0,1.0,714.0,0,2175.0,1,1,0
628,1970961.0,9088.0,10764.0,3411.0,1,58547.0,0,1,1
629,156.0,900.0,0.0,240.0,0,735.0,1,1,0
630,105460.0,22.0,847.0,70.0,0,231440.0,0,1,1


In [18]:
test_df = join_data_id_label_user(None, X_TEST_ID_PATH, TEST_DATA_PATH)
test_df.fillna(0, inplace=True)
test_df

,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,default_profile,translator_type
0,235171,6934,2107,408,1,141268,0,1
1,531,976,10,43613,0,35593,1,1
2,10301,51,47,716,0,1508,1,1
3,139,88,1,46,0,103,1,1
4,191,1609,4,49402,0,40688,0,1
...,...,...,...,...,...,...,...,...
553,15269543,1662,95243,4445,1,353364,0,1
554,623752,2039,1464,51979,1,78610,0,1
555,238571,1207,2366,22377,1,127357,0,1
556,9230728,1737,33459,11728,1,60455,0,1


In [10]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(random_state=2, max_iter=9500).fit(train_df.loc[:, train_df.columns != "label"], train_df.loc[:, "label"])

clf.score(train_df.loc[:, train_df.columns != "label"], train_df["label"])


0.757847533632287

In [11]:
clf.score(dev_df.loc[:, dev_df.columns != "label"], dev_df["label"])

0.740506329113924

In [24]:
pd.DataFrame(clf.predict(test_df)).value_counts()

0    514
1     44
dtype: int64